In [ ]:
import warnings
warnings.filterwarnings("ignore")  # Disable warning messages (Spleeter generates a lot of those :/)

# Music
from autodj.dj import songcollection, tracklister
from autodj.dj.annotators import wrappers
import librosa
import numpy as np

# I/O
import csv
import ipywidgets as w
from ipyfilechooser import FileChooser
from IPython.display import Audio
import matplotlib.pyplot as plt
import os
import soundfile as sf

# Utilities
import demo_features as _f
import demo_util as _u

# Plotting
import plotly.graph_objects as go
import plotly.express as px
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

**Enter the path to your music folder here:**

In [ ]:
music_home_dir = 'PATH/TO/YOUR/MUSIC/DIR'

Song collection loading and annotation widgets:

In [ ]:
sc = songcollection.SongCollection([
    _f.NonstructuralThemeDescriptorWrapper()
])

# Layout
text_output_layout = w.Layout(margin='2px 30px 0px 50px')
button_layout_fillwidth = w.Layout(width='90%',)

# Title for the text output widget
text_output_title = w.HTML(value='<b>Log</b>', layout=text_output_layout)

# Text output widget for info messages
text_output = w.Output(layout=text_output_layout)

# Create and display a FileChooser widget
fc = FileChooser(music_home_dir,
                 select_desc='Load directory', change_desc='Load directory',)
fc.title = '<b>Choose the music folder to load directory</b>'

# Button for loading folder
load_button = fc.children[2].children[0]  # Reuse the load button from the file chooser
load_button.description = 'Load directory'

# Button for loading annotations
annotate_button = w.Button(description='Load song annotations', 
                           button_style='warning', icon='hourglass-half', layout=button_layout_fillwidth)

# Button for clearing song collection
clear_sc_button = w.Button(description='Unload all songs', layout=button_layout_fillwidth)

# Button for clearing the output widget text
clear_text_button = w.Button(description='Clear text output', layout=button_layout_fillwidth)

# Callback for load button
def load_button_callback(b):
    try:
        is_new_dir = fc.selected_path not in sc.directories
        sc.load_directory(fc.selected_path)
        if is_new_dir:
            with text_output:
                print(f'Loaded {fc.selected_path}...')
                print(f'Song collection contains {len(sc.get_annotated())} annotated songs ({len(sc.get_unannotated())} unannotated)')
    except Exception as e:
        with text_output:
            print(f'Select a valid directory first! {fc.selected_path}')
            print(e)
load_button.on_click(load_button_callback)

# Callback for clearing song collection
def clear_sc_button_callback(b):
    sc.clear()
    with text_output:
        print(f'Cleared song collection')
        print(f'Song collection contains {len(sc.get_annotated())} annotated songs ({len(sc.get_unannotated())} unannotated)')
clear_sc_button.on_click(clear_sc_button_callback)

# Callback for clearing text output
def clear_text_button_callback(b):
    text_output.clear_output()
clear_text_button.on_click(clear_text_button_callback)


grid_note = 16

DEFAULT_MARKER_COLOR = 'SteelBlue'
DEFAULT_MARKER_SIZE = 10

ui_songcollection_loader = w.HBox([
        w.VBox((fc, annotate_button, clear_sc_button, clear_text_button)),
        w.VBox((text_output_title, text_output)),
])

In [ ]:
pool = set(sc.get_annotated())
selection = set()
rejection = set()

In [ ]:
def display_song_audio(song):
    
    song.open()
    path_to_audio = song.filepath
    
    y, sr = librosa.load(path_to_audio, sr=44100, offset=30, duration=60)
    y_display = w.Output()
    with y_display:
        display(Audio(y, rate=sr))
        
    return w.HBox((y_display,))

def song_to_str(song):
    return s.title

suggestion_accept = set()
suggestion_reject = set()
suggestion_unsure = set()

btn_suggestion_accept = w.Button(description = 'Accept')
btn_unsuggestion_accept = w.Button(description = 'Accept')
btn_unsure_accept = w.Button(description = 'Accept')
btn_suggestion_reject = w.Button(description = 'Reject')
btn_unsuggestion_reject = w.Button(description = 'Reject')
btn_unsure_reject = w.Button(description = 'Reject')

selectmultiple_pool = w.SelectMultiple(
    options=[(s.title, s) for s in pool],
    rows=10,
    description=f'Library ({len(pool)})',
    layout=w.Layout(width='100%'),
)

selectmultiple_selection = w.SelectMultiple(
    options=[(s.title, s) for s in selection],
    rows=10,
    description=f'Selection ({len(selection)})',
    layout=w.Layout(width='100%'),
)

selectmultiple_rejection = w.SelectMultiple(
    options=[(s.title, s) for s in rejection],
    rows=10,
    description=f'Rejected ({len(rejection)})',
    layout=w.Layout(width='100%'),
)

# Model output browsing
selectmultiple_accept = w.SelectMultiple(
    options=[(s.title, s) for s in suggestion_accept],
    rows=3,
    description=f'Suggested songs',
    layout=w.Layout(width='100%'),
)

selectmultiple_reject = w.SelectMultiple(
    options=[(s.title, s) for s in suggestion_reject],
    rows=3,
    description=f'Unsuggested songs',
    layout=w.Layout(width='100%'),
)

selectmultiple_unsure = w.SelectMultiple(
    options=[(s.title, s) for s in suggestion_unsure],
    rows=3,
    description=f'Unclear?',
    layout=w.Layout(width='100%'),
)

# Buttons
button_add_to_selection = w.Button(
    description='Add to selection',
)
button_add_to_rejection = w.Button(
    description='Add to rejection',
)
button_remove_from_selection = w.Button(
    description='Remove from selection',
)
button_remove_from_rejection = w.Button(
    description='Remove from rejection',
)

button_musicbrowser_pool_play = w.Button(
    description='Play'
)
button_musicbrowser_selection_play = w.Button(
    description='Play'
)
button_musicbrowser_rejection_play = w.Button(
    description='Play'
)
button_musicbrowser_suggestion_play = w.Button(
    description='Play'
)
button_musicbrowser_unsuggestion_play = w.Button(
    description='Play'
)
button_musicbrowser_unsure_play = w.Button(
    description='Play'
)

musicbrowser_output = w.Output()

def update_pool_browser():
    selectmultiple_pool.options = [(s.title, s) for s in pool]
    selectmultiple_pool.description = f'Library ({len(pool)})'
def update_selection_browser():
    selectmultiple_selection.options = [(s.title, s) for s in selection]
    selectmultiple_selection.description = f'Selection ({len(selection)})'
    selectmultiple_rejection.options = [(s.title, s) for s in rejection]
    selectmultiple_rejection.description = f'Rejection ({len(rejection)})'
def update_suggest_browser():
    labeled_songs = selection.union(rejection)
    selectmultiple_accept.options = [(s.title, s) for s in suggestion_accept if s not in labeled_songs]
    selectmultiple_reject.options = [(s.title, s) for s in suggestion_reject if s not in labeled_songs]
    selectmultiple_unsure.options = [(s.title, s) for s in suggestion_unsure if s not in labeled_songs]
    
def annotate_button_callback(b):
    global pool
    
    text_output.clear_output()
    with text_output:
        if len(sc.get_unannotated()) > 0:
            print('Annotating songs...')
            for s in _u.log_progress(sc.get_unannotated()):
                s.annotate()
        else:
            print('All song are already annotated!')
        pool = pool.union(sc.get_annotated())
        update_pool_browser()
annotate_button.on_click(annotate_button_callback)
    
def register_add_to_selection_callback(btn, pool_widget, pool_):
    
    def btn_add_to_selection_callback(b, pool_=pool_):
        global selection
        musicbrowser_output.clear_output()
        with musicbrowser_output:
            selection.update(pool_widget.value)
            pool_ -= set(pool_widget.value)
            
            # Update all browser windows
            update_pool_browser()
            update_selection_browser()
            update_suggest_browser()
    btn.on_click(btn_add_to_selection_callback)
register_add_to_selection_callback(button_add_to_selection, selectmultiple_pool, pool)
register_add_to_selection_callback(btn_suggestion_accept, selectmultiple_accept, suggestion_accept)
register_add_to_selection_callback(btn_unsuggestion_accept, selectmultiple_reject, suggestion_reject)
register_add_to_selection_callback(btn_unsure_accept, selectmultiple_unsure, suggestion_unsure)


def register_add_to_rejection_callback(btn, pool_widget, pool_):
    
    def btn_add_to_rejection_callback(b, pool_=pool_):
        global rejection
        musicbrowser_output.clear_output()
        with musicbrowser_output:
            rejection.update(pool_widget.value)
            pool_ -= set(pool_widget.value)
            
            # Update all browser windows
            update_pool_browser()
            update_selection_browser()
            update_suggest_browser()
    btn.on_click(btn_add_to_rejection_callback)
register_add_to_rejection_callback(button_add_to_rejection, selectmultiple_pool, pool)
register_add_to_rejection_callback(btn_suggestion_reject, selectmultiple_accept, suggestion_accept)
register_add_to_rejection_callback(btn_unsuggestion_reject, selectmultiple_reject, suggestion_reject)
register_add_to_rejection_callback(btn_unsure_reject, selectmultiple_unsure, suggestion_unsure)

def register_remove_from_selection_callback(btn, pool_widget, pool_):
    def btn_remove_from_selection_callback(b, pool_=pool_):
        global pool
        musicbrowser_output.clear_output()
        with musicbrowser_output:
            pool_ -= set(pool_widget.value)
            pool.update(pool_widget.value)
            update_pool_browser()
            update_selection_browser()
    btn.on_click(btn_remove_from_selection_callback)
register_remove_from_selection_callback(button_remove_from_selection, selectmultiple_selection, selection)
register_remove_from_selection_callback(button_remove_from_rejection, selectmultiple_rejection, rejection)

def register_play_audio_callback(btn, pool_widget):
    def btn_pool_play_callback(b):
        musicbrowser_output.clear_output()
        with musicbrowser_output:
            if len(pool_widget.value) > 0:
                display(display_song_audio(pool_widget.value[0]))
    btn.on_click(btn_pool_play_callback)
register_play_audio_callback(button_musicbrowser_pool_play, selectmultiple_pool)
register_play_audio_callback(button_musicbrowser_selection_play, selectmultiple_selection)
register_play_audio_callback(button_musicbrowser_rejection_play, selectmultiple_rejection)
register_play_audio_callback(button_musicbrowser_suggestion_play, selectmultiple_accept)
register_play_audio_callback(button_musicbrowser_unsuggestion_play, selectmultiple_reject)
register_play_audio_callback(button_musicbrowser_unsure_play, selectmultiple_unsure)


import sklearn
model = sklearn.linear_model.LogisticRegression()

button_musicbrowser_train = w.Button(
    description='Train model'
)
button_musicbrowser_clear_model = w.Button(
    description='Clear model'
)

def button_train_callback(b):
    
    def get_label_for_song(s):
        global selection
        return s in selection
        
    with musicbrowser_output:
        print('Preparing data...')
        if len(selection) > len(rejection):
            rejection_sample = np.random.choice(
                list(pool - (selection.union(rejection))), 
                len(selection) - len(rejection))
        else:
            rejection_sample = []
        songs_mini_dataset = selection.union(rejection).union(rejection_sample)
        X = np.array([
            s.simple_song_theme_descriptor.flatten() 
            for s in songs_mini_dataset
        ])
        y = np.array([get_label_for_song(s) for s in songs_mini_dataset]).reshape((-1,1))
        print(X.shape, y.shape)
        print('Training model...')
        model.fit(X, y)
        print('Model trained!')
        
        suggestion_accept.clear()
        suggestion_reject.clear()
        suggestion_unsure.clear()
    
        # Do some suggestions
        annotated_songs = selection.union(rejection)
        pool_list = np.array(list(pool))                             
        X_test = np.array([
            s.simple_song_theme_descriptor.flatten() 
            for s in pool_list if s not in annotated_songs
        ])
        pred = model.predict_log_proba(X_test)
        pred_args = np.argsort(pred[:,1])
        pred_top_10 = pred_args[-10:]
        pred_bottom_10 = pred_args[:10]
        suggestion_accept.update(list(pool_list[pred_top_10]))
        suggestion_reject.update(list(pool_list[pred_bottom_10]))
        
        print(np.exp(pred[pred_top_10]))
        print(np.exp(pred[pred_bottom_10]))
#         for i, (p, s) in enumerate(zip(pred[:,1], pool_list)):
#             print(s.title, p)
#             if p > np.log(0.8):
#                 suggestion_accept.add(s)
#             elif p < np.log(0.2):
#                 suggestion_reject.add(s)
#             else:
#                 suggestion_unsure.add(s)
        update_suggest_browser()
button_musicbrowser_train.on_click(button_train_callback)


ui_songcollection_browser = w.VBox([
    w.HBox([
        selectmultiple_pool,
        w.VBox([
            button_add_to_selection,
            button_add_to_rejection,
            button_musicbrowser_pool_play,
        ]),
    ]),
    w.HBox([
        selectmultiple_selection,
        w.VBox([
            button_remove_from_selection,
            button_musicbrowser_selection_play,
        ]),
    ]),
    w.HBox([
        selectmultiple_rejection,
        w.VBox([
            button_remove_from_rejection,
            button_musicbrowser_rejection_play,
        ]),
    ]),
    button_musicbrowser_train,
    w.VBox([
        w.HBox([
            selectmultiple_accept,
            w.VBox([btn_suggestion_accept, btn_suggestion_reject, button_musicbrowser_suggestion_play]),
        ]),
        w.HBox([
            selectmultiple_reject,
            w.VBox([btn_unsuggestion_accept, btn_unsuggestion_reject, button_musicbrowser_unsuggestion_play]),
        ]),
        w.HBox([
            selectmultiple_unsure,
            w.VBox([btn_unsure_accept, btn_unsure_reject, button_musicbrowser_unsure_play]),
        ])
    ]),
    musicbrowser_output,
])

Song collection browsing code:

In [ ]:
tab = w.Tab([ui_songcollection_loader, ui_songcollection_browser])
tab.set_title(0, 'Loading songs')
tab.set_title(1, 'Music browser')
display(tab)